In [1]:
# Nuestro path de inicio siempre debe ser el home dir, e
import os
os.chdir(os.path.abspath(".."))

# Imports
import json
import pandas as pd
import stable_whisper
import warnings
from src.dataload import audio_select_mean_vote

# ignoramos warnings
warnings.filterwarnings('ignore')

In [3]:
# Archivos importantes y paths
df_annotations = pd.read_excel('data/annotations.xlsx')

# Omitimos todos los audios en development
df_annotations = df_annotations[df_annotations['Type'] != 'Development'].reset_index(drop = True)

In [4]:
# Obtenemos el json con los intervalos
with open('data/intervalos_transcripciones.json', 'r') as f:
    intervalos_transcripciones = json.load(f)
    
# Comprabamos seguimos teniendo los 244 audios
len(intervalos_transcripciones.keys())

244

In [41]:
# Audios que no estan en development
lista_audios = df_annotations['Audio_Name'].unique()
len(lista_audios)

205

In [84]:
# Creo mi diccionario que voy a guardare como json
targets_mean_vote = {}

# Loop por cada audio en la lista de audios que no estan en development
for audio_name in lista_audios:
    
    print('Procesando el audio:', audio_name)
    
    df_audio = df_annotations[df_annotations['Audio_Name'] == audio_name]
    partes = df_audio['Part_Num'].max()
    
    # Inicializo valores para el audio en el diccionario
    targets_mean_vote[audio_name] = {}
    targets_mean_vote[audio_name]['rangos'] = []
    targets_mean_vote[audio_name]['targets'] = []
    
    # Loop por cada parte del audio
    for part_num in range(partes):
        part_num = part_num + 1
        
        print(f'Procesando parte {part_num} de {audio_name}')
        
        # Obtengo votación para la parte del audio
        df_votacion = audio_select_mean_vote(df_annotations, audio_name = audio_name, part_num = part_num)
        
        # Verificamos que solo halla un start time y nos lo quedamos
        assert len(df_audio[df_audio['Part_Num'] == part_num]['start_time'].unique()) == 1, 'Más de un start time'
        start_time = df_audio[df_audio['Part_Num'] == part_num]['start_time'].unique()[0]
        
        # Agregamos al audio el start time necesario
        df_votacion['Time'] = df_votacion['Time'] + start_time
        
        # Loop principal para cargar las votaciones promedio
        i = 0
        
        int_act = intervalos_transcripciones[audio_name] 
        ts_primera_anotacion = df_votacion['Time'][0]
        ts_ultima_anotacion = df_votacion['Time'][len(df_votacion) - 1]
        
        while i + 1 <= len(int_act) and int_act[i][1] < ts_ultima_anotacion:
            
            if int_act[i][0] > ts_primera_anotacion:
                inicio = intervalos_transcripciones[audio_name][i][0]
                fin = intervalos_transcripciones[audio_name][i][1]
                
                votacion_promedio = df_votacion[(df_votacion['Time'] >= inicio) & (df_votacion['Time'] <= fin)][['Valence','Arousal','Dominance']].mean().values.tolist()
                
                targets_mean_vote[audio_name]['targets'].append(votacion_promedio)
                targets_mean_vote[audio_name]['rangos'].append((inicio, fin))
            
            i += 1
            
    assert len(targets_mean_vote[audio_name]['targets']) == len(targets_mean_vote[audio_name]['rangos']), 'Rangos no coinciden'
        
    print(f'{audio_name} procesado con éxito')
    print('')
    print('---------------------------------------------------------')

Procesando el audio: MSP-Conversation_0021.wav
Procesando parte 1 de MSP-Conversation_0021.wav
Procesando parte 2 de MSP-Conversation_0021.wav
Procesando parte 3 de MSP-Conversation_0021.wav
Procesando parte 4 de MSP-Conversation_0021.wav
MSP-Conversation_0021.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0023.wav
Procesando parte 1 de MSP-Conversation_0023.wav
Procesando parte 2 de MSP-Conversation_0023.wav
MSP-Conversation_0023.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0035.wav
Procesando parte 1 de MSP-Conversation_0035.wav
Procesando parte 2 de MSP-Conversation_0035.wav
MSP-Conversation_0035.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0043.wav
Procesando parte 1 de MSP-Conversation_0043.wav
Procesando parte 2 de MSP-Conversation_0043.wav
MSP-Conversation_

Procesando parte 2 de MSP-Conversation_0166.wav
MSP-Conversation_0166.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0167.wav
Procesando parte 1 de MSP-Conversation_0167.wav
Procesando parte 2 de MSP-Conversation_0167.wav
MSP-Conversation_0167.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0172.wav
Procesando parte 1 de MSP-Conversation_0172.wav
Procesando parte 2 de MSP-Conversation_0172.wav
MSP-Conversation_0172.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0180.wav
Procesando parte 1 de MSP-Conversation_0180.wav
Procesando parte 2 de MSP-Conversation_0180.wav
MSP-Conversation_0180.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0183.wav
Procesando parte 1 de MSP-Conversation_0183.wav
Procesan

Procesando parte 2 de MSP-Conversation_0397.wav
MSP-Conversation_0397.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0401.wav
Procesando parte 1 de MSP-Conversation_0401.wav
Procesando parte 2 de MSP-Conversation_0401.wav
Procesando parte 3 de MSP-Conversation_0401.wav
Procesando parte 4 de MSP-Conversation_0401.wav
MSP-Conversation_0401.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0417.wav
Procesando parte 1 de MSP-Conversation_0417.wav
Procesando parte 2 de MSP-Conversation_0417.wav
Procesando parte 3 de MSP-Conversation_0417.wav
Procesando parte 4 de MSP-Conversation_0417.wav
MSP-Conversation_0417.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0418.wav
Procesando parte 1 de MSP-Conversation_0418.wav
Procesando parte 2 de MSP-Conversation_0418.wav
MSP-Conversation

Procesando parte 2 de MSP-Conversation_0646.wav
MSP-Conversation_0646.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0672.wav
Procesando parte 1 de MSP-Conversation_0672.wav
Procesando parte 2 de MSP-Conversation_0672.wav
Procesando parte 3 de MSP-Conversation_0672.wav
MSP-Conversation_0672.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0675.wav
Procesando parte 1 de MSP-Conversation_0675.wav
Procesando parte 2 de MSP-Conversation_0675.wav
Procesando parte 3 de MSP-Conversation_0675.wav
MSP-Conversation_0675.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0678.wav
Procesando parte 1 de MSP-Conversation_0678.wav
Procesando parte 2 de MSP-Conversation_0678.wav
Procesando parte 3 de MSP-Conversation_0678.wav
Procesando parte 4 de MSP-Conversation_0678.wav
MSP-Conversation

Procesando parte 2 de MSP-Conversation_0858.wav
MSP-Conversation_0858.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0869.wav
Procesando parte 1 de MSP-Conversation_0869.wav
Procesando parte 2 de MSP-Conversation_0869.wav
MSP-Conversation_0869.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0878.wav
Procesando parte 1 de MSP-Conversation_0878.wav
Procesando parte 2 de MSP-Conversation_0878.wav
MSP-Conversation_0878.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0883.wav
Procesando parte 1 de MSP-Conversation_0883.wav
Procesando parte 2 de MSP-Conversation_0883.wav
MSP-Conversation_0883.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0893.wav
Procesando parte 1 de MSP-Conversation_0893.wav
Procesan

Procesando parte 2 de MSP-Conversation_1258.wav
Procesando parte 3 de MSP-Conversation_1258.wav
Procesando parte 4 de MSP-Conversation_1258.wav
MSP-Conversation_1258.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1353.wav
Procesando parte 1 de MSP-Conversation_1353.wav
Procesando parte 2 de MSP-Conversation_1353.wav
MSP-Conversation_1353.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1356.wav
Procesando parte 1 de MSP-Conversation_1356.wav
Procesando parte 2 de MSP-Conversation_1356.wav
Procesando parte 3 de MSP-Conversation_1356.wav
Procesando parte 4 de MSP-Conversation_1356.wav
MSP-Conversation_1356.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1406.wav
Procesando parte 1 de MSP-Conversation_1406.wav
Procesando parte 2 de MSP-Conversation_1406.wav
MSP-Conversation

Procesando parte 4 de MSP-Conversation_1752.wav
MSP-Conversation_1752.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1870.wav
Procesando parte 1 de MSP-Conversation_1870.wav
Procesando parte 2 de MSP-Conversation_1870.wav
Procesando parte 3 de MSP-Conversation_1870.wav
MSP-Conversation_1870.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1884.wav
Procesando parte 1 de MSP-Conversation_1884.wav
Procesando parte 2 de MSP-Conversation_1884.wav
MSP-Conversation_1884.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1890.wav
Procesando parte 1 de MSP-Conversation_1890.wav
Procesando parte 2 de MSP-Conversation_1890.wav
MSP-Conversation_1890.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1892.wav
Procesan

Procesando parte 2 de MSP-Conversation_2253.wav
Procesando parte 3 de MSP-Conversation_2253.wav
MSP-Conversation_2253.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_2277.wav
Procesando parte 1 de MSP-Conversation_2277.wav
Procesando parte 2 de MSP-Conversation_2277.wav
Procesando parte 3 de MSP-Conversation_2277.wav
Procesando parte 4 de MSP-Conversation_2277.wav
MSP-Conversation_2277.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_2279.wav
Procesando parte 1 de MSP-Conversation_2279.wav
Procesando parte 2 de MSP-Conversation_2279.wav
Procesando parte 3 de MSP-Conversation_2279.wav
Procesando parte 4 de MSP-Conversation_2279.wav
MSP-Conversation_2279.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_2281.wav
Procesando parte 1 de MSP-Conversation_2281.wav
Procesando parte

In [85]:
# Guardo el diccionario en archivo json
with open('data/targets_mean_vote.json', 'w') as f:
    json.dump(targets_mean_vote, f)